# FAISS With Haystack

FAISS is unfortunately **not** presently supported on Windows, so if you are on Windows then you will need to stick with Elasticsearch. If you have access to Linux or Mac then continue.

We'll be using Haystack again, so fortunately setup is very straight-forward. We first import and initialize a FAISS document store using a very similiar logic to what we used before - but this time we will be storing the FAISS index locally.

Storing the index locally means that we will need two files, a SQLite database, and the FAISS index. We create the FAISS index later, but we create the SQLite database on initialization.

We will store both in the `models` directory, but adjust this to your own needs.

In [1]:
path = '../../models/faiss'

import os

if not os.path.exists(path):
    os.makedirs(path)

And now we include this path within a SQLite database location string in the following document store initialization.

In [2]:
from haystack.document_store.faiss import FAISSDocumentStore

# initialize FAISS
document_store = FAISSDocumentStore(
    faiss_index_factory_str='Flat',
    sql_url=f'sqlite:///{path}/squad_dev.db',
    return_embedding=True
)

03/14/2022 09:11:08 - INFO - faiss -   Loading faiss.


Next, we load our validation data from file, which we will be adding to the FAISS index.

In [3]:
import json

with open('../../data/squad/dev.json', 'r') as f:
    squad = json.load(f)

## Adding Data

As we saw with Elasticsearch, our current FAISS index has been initialized but contains nothing. Now we need to populate the index with our *dev.json* data. 

This time, we'll be making use of the Haystack `Document` object. Which we import with:

In [4]:
from haystack import Document

This object prepares our data into the correct object format for our document stores - which in this case is FAISS.

As before where we had a dictionary with two keys `'text'` and `'meta'`, the *Document* object provides two corresponding arguments, `text` and `meta`. So rather than using the format we used before which looked like:

```json
{
    'text': '<document text here>',
    'meta': {
        'other': '<other info here>'
    }
}
```

We will be using this *Document* object format instead:

```python
Document(
    text='<document text here>',
    meta={
        'other': '<other info here>'
    }
)
```

Just like before, we will be feeding these *Document* objects into a list, which we will then feed into our FAISS `write_documents` method. Remember, our dataset contains duplicate contexts, so we must remove them first using `list(set(...))`.

In [5]:
# create list of contexts
contexts = [sample['context'] for sample in squad]

# remove duplicates
contexts = list(set(contexts))

# create list of Document objects
squad_docs = [Document(text=sample) for sample in contexts]

Now, because we're storing our FAISS index on file, we may find (if running this script more than once) that we first need to delete any documents that already exist in the index.

In [6]:
document_store.delete_all_documents()

Then we add the data to the index just like before:

In [7]:
document_store.write_documents(squad_docs)

The way that our documents are indexed will depend on the embedding model being used by our retriever. So, we need to initialize our DPR model (the retriever), and then `update_embeddings` using this retriever.

In [8]:
from haystack.retriever.dense import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

document_store.update_embeddings(retriever=retriever)

03/14/2022 09:13:54 - INFO - filelock -   Lock 140455006151392 acquired on /Users/adenevreze/.cache/huggingface/transformers/52774638a790c9ebc5ce11005b260f79cd4cc389abdab9eaa31e8f09d15b4f46.13b559f49587470ab6d85a7dde13174670a0b61c1b942d1489c96023f5d03772.lock
Downloading: 100%|██████████| 493/493 [00:00<00:00, 90.8kB/s]
03/14/2022 09:13:55 - INFO - filelock -   Lock 140455006151392 released on /Users/adenevreze/.cache/huggingface/transformers/52774638a790c9ebc5ce11005b260f79cd4cc389abdab9eaa31e8f09d15b4f46.13b559f49587470ab6d85a7dde13174670a0b61c1b942d1489c96023f5d03772.lock
03/14/2022 09:13:58 - INFO - filelock -   Lock 140455005849728 acquired on /Users/adenevreze/.cache/huggingface/transformers/41dac75f5df9070331cb0e4bf318c9fdeaef38d9ffd8ca80993c7db830d0c674.446ee898f4788c3ee90f8e7ee5a50281905f509e698f76dc0b583eb74ef973bd.lock
Downloading: 100%|██████████| 438M/438M [03:09<00:00, 2.32MB/s] 
03/14/2022 09:17:08 - INFO - filelock -   Lock 140455005849728 released on /Users/adenevreze/

Now that we've fully prepared our document store, we can save it. We will save to the same location we saved our SQLite database, but this time we will be using the *.faiss* filetype.

In [9]:
document_store.save(f'{path}/squad_dev.faiss')

Our FAISS index is now saved to file! We'll go ahead and delete the `document_store` and `retriever`, and try reinitializing both using the data we've saved to file.

In [10]:
del document_store, retriever

All we do now is apply the `load` method directly from `FAISSDocumentStore`, including both the FAISS index location, and SQLite database location:

In [11]:
document_store = FAISSDocumentStore.load(f'{path}/squad_dev.faiss', f'sqlite:///{path}/squad_dev.db')

And now we can re-initialize our retriever, using the same arguments as before.

In [12]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

Finally, we can begin retrieving relevant contexts to our questions using `retriever.retrieve`, which requires a single argument, `query`.

In [13]:
retriever.retrieve('What subject is most abstract?')

Creating Embeddings: 100%|██████████| 1/1 [00:00<00:00,  5.42 Batches/s]


[{'text': "A Turing machine is a mathematical model of a general computing machine. It is a theoretical device that manipulates symbols contained on a strip of tape. Turing machines are not intended as a practical computing technology, but rather as a thought experiment representing a computing machine—anything from an advanced supercomputer to a mathematician with a pencil and paper. It is believed that if a problem can be solved by an algorithm, there exists a Turing machine that solves the problem. Indeed, this is the statement of the Church–Turing thesis. Furthermore, it is known that everything that can be computed on other models of computation known to us today, such as a RAM machine, Conway's Game of Life, cellular automata or any programming language can be computed on a Turing machine. Since Turing machines are easy to analyze mathematically, and are believed to be as powerful as any other model of computation, the Turing machine is the most commonly used model in complexity 

And now we've extracted a few contexts stored within FAISS, that our DPR model believes answers our query.

* * *

In [14]:
retriever.retrieve('What was the french revolution?')

Creating Embeddings: 100%|██████████| 1/1 [00:00<00:00,  5.74 Batches/s]


[{'text': "The French and Indian War (1754–1763) was the North American theater of the worldwide Seven Years' War. The war was fought between the colonies of British America and New France, with both sides supported by military units from their parent countries of Great Britain and France, as well as Native American allies. At the start of the war, the French North American colonies had a population of roughly 60,000 European settlers, compared with 2 million in the British North American colonies. The outnumbered French particularly depended on the Indians. Long in conflict, the metropole nations declared war on each other in 1756, escalating the war from a regional affair into an intercontinental conflict.", 'id': '529a9f14-d32a-4eb0-b6e4-a972324c46fe', 'score': 68.85273, 'probability': 0.665639236818449, 'question': None, 'meta': {'vector_id': '403'}, 'embedding': None},
 {'text': 'Louis XIV gained the throne in 1643 and acted increasingly aggressively to force the Huguenots to conv